In [106]:
import numpy as np
import tensorflow as tf
session = tf.Session()

In [107]:
data_size = 25
data_1d = np.random.normal(size = data_size) # se definen los datos como un vector de tamaño 25
x_data = tf.placeholder(shape=[data_size], dtype=tf.float32) # se define el placeholder

#### en el caso de imagenes, tensorflow trabaja con 4 dimensiones, las cuales son:
      [cantidad de elementos, anchura, altura, canales de color]

# capa de convolucion
    paso 1: transforma el vector de 1d en un vector de 4d
    paso 2: ejecuta la operacion de convolucion
    paso 3: realiza un squeeze para aplanar el resultado

In [108]:
def conv_layer_1d(input_1d, my_filter):
    input_2d = tf.expand_dims(input_1d, 0) # se añade 1 dimension
    input_3d = tf.expand_dims(input_2d, 0) # se añade 1 dimension
    input_4d = tf.expand_dims(input_3d, 3) # se añade 1 dimension
    convolution = tf.nn.conv2d(input_4d, filter=my_filter, strides=[1,1,1,1], padding="VALID") # se ejecuta la operaciond e convolucion
    output = tf.squeeze(convolution) # se quita las dimensiones añadidas anteriormente
    return(output)

In [109]:
my_filter = tf.Variable(tf.random_normal(shape=[1,5,1,1])) #[cantidad de elementos, anchura, altura, canales de color]
my_conv_output = conv_layer_1d(x_data, my_filter)

# capa de activacion

In [110]:
def activation(input_1d):
    return tf.nn.relu(input_1d)

In [111]:
my_activation_output = activation(my_conv_output)

# capa de maxpool
    -se encarga de eliminar valores mediocres
    -devuleve el valor maximo de los elementos que caen dentro de la ventana

In [112]:
def max_pool(input_1d, width): #"width" es el tamaño de la ventana
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    pooling = tf.nn.max_pool(input_4d, ksize=[1,1,width,1], strides=[1,1,1,1], padding="VALID")
    output = tf.squeeze(pooling) # se quita las dimensiones añadidas anteriormente
    return(output)

In [113]:
my_maxpool_output = max_pool(my_activation_output, width=5) # en este caso el tamaño de la ventana sera de 5

# capa de fullyconnected
    -conecta toda la informacion anterior y devuleve la salida
    -la salida se debe indicar, segun lo que necesitamos

In [114]:
def fully_connected(input_layer, num_output):
    weight_shape = tf.squeeze(tf.stack([tf.shape(input_layer), [num_output]])) # se define el tamaño de la matriz de pesos
    weight = tf.random_normal(weight_shape, stddev=0.1) # matriz de pesos con el tamaño definido anteirormente y una desviacion estandar pequeña
    bias = tf.random_normal(shape=[num_output]) # se define el bias
    input_layer_2d = tf.expand_dims(input_layer, 0) # se expande la dimension del input_layer a 2d para poder multiplicar
    full_output = tf.add(tf.matmul(input_layer_2d, weight), bias) # operacion
    full_output_1d = tf.squeeze(full_output) #se quita las dimensiones extras
    return(full_output_1d)

In [115]:
my_full_output = fully_connected(my_maxpool_output, 5) # el valor de la salida en este caso sera 5

### hasta aqui la definicion de capas

# se inicializan variables 

In [116]:
init = tf.global_variables_initializer()
session.run(init)

# resultados de cada capa

In [117]:
# datos de entrada iniciales
data_1d

array([ 0.82776061, -0.37962648, -0.90966925, -0.4982885 , -0.91240613,
        0.93977772,  0.90419787,  0.59773432, -0.23843811, -1.11869032,
       -0.12169182,  0.90916853, -0.49330608,  1.15427104, -0.17326395,
        0.78375885, -0.23922061,  0.24791774, -1.03347151, -0.94509943,
       -1.37755911,  0.18543963,  0.26871107,  0.37879331,  0.7138591 ])

In [118]:
# el filtro
session.run(my_filter)

array([[[[1.4869758 ]],

        [[0.21947595]],

        [[1.0534577 ]],

        [[0.5079911 ]],

        [[0.75944173]]]], dtype=float32)

In [119]:
# capa de convolucion (multiplica el my_filter con los datos de data_1d)
print("INPUT: tamaño 25(correspondiente a data_1d)")
print("OPERACION: capa de convolucion con filtro de tamaño 5 + stride de tamaño 1")
print("SALIDA: tamaño 21")
print(session.run(my_conv_output, feed_dict={x_data: data_1d}))

INPUT: tamaño 25(correspondiente a data_1d)
OPERACION: capa de convolucion con filtro de tamaño 5 + stride de tamaño 1
SALIDA: tamaño 21
[-0.75680244 -1.0388596  -1.2591157   0.9620908  -0.07536905  1.2548596
  0.56382155  0.2866342  -0.64106226 -0.10639685 -0.04631376  2.9668212
 -0.44625732  2.5707598  -0.9965533   0.13135588 -2.9162998  -1.4127538
 -2.897102   -1.088154   -0.9900638 ]


In [120]:
# capa de activacion (elimina los valores negativos de la salida devuelta por la capa de convolucion)
print("INPUT: tamaño 21(correspondiente a la salida de la capa my_conv_output)")
print("OPERACION: se aplica funcion relu a los valores de la salida de la capa my_conv_output")
print("SALIDA: tamaño 21")
print(session.run(my_activation_output, feed_dict={x_data: data_1d}))

INPUT: tamaño 21(correspondiente a la salida de la capa my_conv_output)
OPERACION: se aplica funcion relu a los valores de la salida de la capa my_conv_output
SALIDA: tamaño 21
[-0.         -0.         -0.          0.9620908  -0.          1.2548596
  0.56382155  0.2866342  -0.         -0.         -0.          2.9668212
 -0.          2.5707598  -0.          0.13135588 -0.         -0.
 -0.         -0.         -0.        ]


In [121]:
# capa de maxpool(toma en fila valores de 5 en 5 y elije el mas grande)
print("INPUT: tamaño 21(correspondiente a la salida de la capa de activacion)")
print("OPERACION: maxpooling con ventana de tamaño 5 + stride de tamaño 1")
print("SALIDA: tamaño 17")
print(session.run(my_maxpool_output, feed_dict={x_data: data_1d}))

INPUT: tamaño 21(correspondiente a la salida de la capa de activacion)
OPERACION: maxpooling con ventana de tamaño 5 + stride de tamaño 1
SALIDA: tamaño 17
[ 0.9620908   1.2548596   1.2548596   1.2548596   1.2548596   1.2548596
  0.56382155  2.9668212   2.9668212   2.9668212   2.9668212   2.9668212
  2.5707598   2.5707598   0.13135588  0.13135588 -0.        ]


In [122]:
# capa fully_conected (conecta toda la informacion anterior y devuleve la salida)
print("INPUT: tamaño 17(correspondiente a la salida de la capa de max_pool)")
print("OPERACION: conectar la entrada data_1d con los 5 valores de salida")
print("SALIDA: tamaño 5")
print(session.run(my_full_output, feed_dict={x_data: data_1d}))

INPUT: tamaño 17(correspondiente a la salida de la capa de max_pool)
OPERACION: conectar la entrada data_1d con los 5 valores de salida
SALIDA: tamaño 5
[-1.287904   -0.41955012 -0.92214954 -2.7925303   0.1916806 ]
